# LMBR Notes

## Levenberg-Marquardt (LM) Algorithm

### 1) Basic Gradient Descent:
$$
\begin{align}
\mathbf{x_{n+1}} &= \mathbf{x_n} - \lambda \nabla f( \mathbf{x_n} )
\end{align}
$$

### 2) Newton-Raphson (NR) method:

2a) The solution for the stationary point of f(x):
$$
\begin{align}
\nabla f( \mathbf{x_{n+1}} ) &= \nabla f( \mathbf{x_n} ) + (\mathbf{x_{n+1}} - \mathbf{x_n})^T \nabla^2 f( \mathbf{x_n} ) +
\text{Higher order terms = 0}
\end{align}
$$

2b) Neglecting the higher-order terms, we get:
$$
\begin{align}
\mathbf{x_{n+1}} &= \mathbf{x_n} - ( \nabla^2 f( \mathbf{x_n} ) )^{-1} \nabla f( \mathbf{x_n} )
\end{align}
$$

This method uses the inverse of the Hessian matrix $H = \nabla^2 f( \mathbf{x_n} )$ (i.e., the function's curvature) to reach the minimum point.

Drawbacks of the NR method include:
* Convergence depends strongly on the initial guess. Starting in a low-gradient (i.e., flat) region of the loss function will likely impede optimization.
* The Hessian matrix must be positive-definite and not singular. Otherwise, the search direction may result in gradient ascent, or the algorithm stops functioning entirely.
* Calculation of the Hessian matrix and its inverse is very time-consuming for large-dimension $\mathbf{x}$ vectors.

### 3) Levenberg-Marquardt (LM) Optimization:

$$
\begin{align}
\mathbf{x_{n+1}} &= \mathbf{x_n} - ( \nabla^2 f( \mathbf{x_n} ) + \lambda I )^{-1} \nabla f( \mathbf{x_n} )
\end{align}
$$

This method combines basic gradient descent and NR optimization.
* When $\lambda \rightarrow 0$, the LM method approaches the NR method.
* When $\lambda \rightarrow \infty$, the LM method approaches basic gradient descent with a small learning rate (step size) of $\lambda^{-1}$.

The gain, $\lambda$, ensures that $\lambda I + H$ is positive-definite (thus avoiding gradient ascent), even if $H$ is not.

In each update step:
* If loss decreases (i.e., gradient descent), then information from $H$ is helping, so $\lambda$ is reduced (usually by a factor of 10).
* If loss increases (i.e., gradient ascent), then information from $H$ is not helping. Thus, the step is rejected, $\lambda$ is increased (usually by a factor of 10), and the step is recalculated in such fashion until loss decreases.

For small optimization problems, LM optimization is faster than basic gradient descent and more stable than NR optimization.

## LM Optimization with Bayesian Regularization

Source: "Gauss-Newton Approximation to Bayesian Learning" by Foresee and Hagan, 1997.

## 1) Loss Function and Objective Function Parameters

The objective is to minimize both $E_D$ and $E_W$, where $E_D$ is the sum of squared errors between network responses and training data, $D$, while $E_W$ is the sum of squares of network weights, $W$. Such regularization should produce accurate, generalized nonlinear regressions.

Thus, the objective function is $F = \beta E_D + \alpha E_W $, where the initial values of $\alpha$ and $\beta$ are hyperparameters.
* If $\beta >> \alpha$, then optimization will focus on minimizing errors.
* If $\beta << \alpha$, then optimization will focus on minimizing weight sizes.

## 2) Initialization

The authors chose to use initial values of $\alpha = 0$ and $\beta = 1$, and to initialize weights via the Nguyen-Widrow method (1990).

## 3) Training

1) Take one step of LM optimization to minimize the objective function $F(W) = \beta E_D + \alpha E_W$. 

2) Compute the effective number of parameters $\gamma = N - 2 \alpha tr(H^{-1})$, where N is the total number of parameters in the network. \
   $\gamma$ measures how many parameters in the network are effectively used in reducing the error function. \
   Make use of the Gauss-Newton approximation of $H$ available in the LM algorithm:
   $H = \nabla^2 F(W) \approx 2 \beta J^T J + 2 \alpha I_N$, where $J$ is the Jacobian matrix of the training set errors.

3) Compute new estimates for the objective function parameters:
$$
\begin{align}
\alpha &= \frac{\gamma}{2 E_W(W)} \\
\beta &= \frac{n - \gamma}{2 E_D(W)}
\end{align}
$$ \
where $n$ is the total number of training data. \
One alternate way to calculate $\alpha$ and avoid iteration deficiency is:
$$
\begin{align}
\alpha &= \frac{N}{2 E_W + tr(H^{-1})}
\end{align}
$$


4) Repeat steps 1 - 3 until convergence.